In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

In [ ]:
import re
import requests
import numpy as np
import pandas as pd
import pickle
from google.cloud import storage
from google.cloud import speech
from google.cloud import speech_v1p1beta1 as speech
import io
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'cred/translate-cred.json'

In [ ]:
pd.set_option('display.max_colwidth', -1)
pd.options.display.max_rows = 300

## Transcribe episodes for select podcasts

In [ ]:
# Select the podcasts and episodes to download
podcast_id = '1295839054'
podcast_artwork = 'https://is5-ssl.mzstatic.com/image/thumb/Podcasts123/v4/bf/04/aa/bf04aaef-323a-ffa5-eda0-8db690b75310/mza_5471750803414788155.jpeg/626x0w.jpg'
episode_title = "L'immigration nuit-elle à l'économie ?"
episode_link = 'https://rss.art19.com/episodes/ddeb0407-3097-4dae-ba4d-1b11b651438e.mp3'
epidode_date = '2017-11-26'
episode_lang = 'fr-FR'

In [ ]:
# Download episode
def download_episode(podcast_id, episode_link, download_dir):
    # Construct file names
    file_format = re.sub('\?.*$', '', episode_link.split('.')[-1])
    file_name = "{}.{}".format(podcast_id, file_format)
    file_path = "{}/{}".format(download_dir, file_name)

    # Download
    os.system("rm {}".format(file_path))
    os.system("wget -O {} {}".format(file_path, episode_link))
    return file_path

In [ ]:
file_path = download_episode(podcast_id, episode_link, 'data/podcasts')

In [ ]:
# Convert mp3 to wav
def mp3_to_wav(file_path):
    if file_path.split('.')[1] == 'mp3':
        file_path_wav = re.sub('\.mp3$', '.wav', file_path)
        os.system("../../ffmpeg/ffmpeg -i {} -ac 1 {}".format(file_path, file_path_wav))
        return file_path_wav
    return file_path

In [ ]:
file_path = mp3_to_wav(file_path)

In [ ]:
# Upload the audio file to Google cloud storage
def upload_file_to_gcs(bucket_name, file_path):
    storage_client = storage.Client()
    
    # Get bucket
    bucket = storage_client.get_bucket(bucket_name)
    
    # Upload
    file_name = file_path.split('/')[-1]
    blob = bucket.blob(file_name)
    blob.upload_from_filename(file_path)
    
    gcs_uri = "gs://{}/{}".format(bucket_name, file_name)
    return gcs_uri

In [ ]:
gcs_uri = upload_file_to_gcs('tingshuo', file_path)

In [ ]:
# Run ASR
def run_asr(gcs_uri, src_lang):
    asr_client = speech.SpeechClient()
    audio = speech.types.RecognitionAudio(uri=gcs_uri)
    
    config = speech.types.RecognitionConfig(
        encoding=speech.enums.RecognitionConfig.AudioEncoding.LINEAR16,
        language_code=src_lang,
        enable_automatic_punctuation=True)
    
    operation = asr_client.long_running_recognize(config, audio)
    response = operation.result(timeout=3600*2)
    return response

In [ ]:
response = run_asr(gcs_uri, episode_lang)

In [ ]:
# Output transcript
for result in response.results:
    with open(re.sub('\.wav$', '.txt', file_path), 'a') as f:
        f.write(result.alternatives[0].transcript + '\n')